To run this project, `ollama` must be installed. This is a package that allows for LLM usage, easily. 

The `ollama` server can be run using `ollama serve`, for testing purposes. 

In [14]:
! pip3 install ollama streamlit

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 8.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 5.6 MB/s eta 0:00:0000:0100:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 8.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 8.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 8.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 7.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
from httpx import ConnectError
import ollama
import typing
import subprocess
import streamlit as st
# Check if models are installed or not.
try:
    print("Installed models on local ollama instance:")
    # print(ollama.list())
    INSTALLEDMODELS: list[str] = []
    for modelName in ollama.list()["models"]:
        print(modelName["name"])
        INSTALLEDMODELS.append(modelName["name"])
except ConnectError:
    raise Exception("ollama server is not online. \nUse ollama serve to run the ollama daemon.")

Installed models on local ollama instance:
phi3:mini


Papers generated use any notes included in the `NOTEFILE`. 

__A mock paper can be generated by__
- supplying notes(required), 
- supplying an LLM model name(default will be `phi3:mini` ), 
- providing the number of questions(between 2 and 5), 
- the total number of marks that the questions should total up to(default will be 100), 
- the question type(short-form questions, multiple choice questions(default), essay based questions)

In [5]:
"""Check defaults and make sure there are no exceptions. """

MODELNAME: str = "phi3:mini"

if MODELNAME not in INSTALLEDMODELS:
    raise Exception(f"Model name {MODELNAME} not in installed models.")
else:
    ollama.pull(MODELNAME)

NOTEFILE: str = "notes.txt"

try:
    with open(NOTEFILE, "r") as questionTemplate:
        NOTES: str = "".join(questionTemplate.readlines())
except FileNotFoundError:
    raise Exception(f"Could not find {NOTEFILE}.")


QUESTIONNUMBER: int = 5

if QUESTIONNUMBER > 5 or QUESTIONNUMBER < 2:
    raise Exception("Question amount not suitable.")

TOTALMARKS: int = 10

if TOTALMARKS // QUESTIONNUMBER < 1:
    raise Exception("Number of marks is too low.")

QUESTIONTYPE: str = "long"
""" This can differ between short, long and mcq."""

# try:
#     with open("exampleQuestions.txt", "r") as QUESTIONSFILE:
#         EXAMPLEQUESTIONS = "".join(QUESTIONSFILE.readlines())
# except OSError:
#     raise Exception("Question file does not exist.")

""" If true, then questions can be divided into a,b,c or i, ii, iii.
    Question subdivision support can be added later. 
"""
# if QUESTIONNUMBER > 10:
#     SUBDIVISIONS = True
# else:
#     SUBDIVISIONS = False

' If true, then questions can be divided into a,b,c or i, ii, iii.\n    Question subdivision support can be added later. \n'

In [6]:
class QuestionType:
    """This class houses the different types of questions, and the prompts, that can be created by the program for each paper.
    Parameters:
    name: str - the name of the question type.
    systemPrompt: str - the initial prompt that should be added to generate the questions.
    examplePrompt: str - any example questions that would aid in generation. 
    """

    def __init__(self, name: str, systemPrompt: str, examplePrompt: str = None) -> None:
        self.name = name
        self.systemPrompt = systemPrompt
        self.examplePrompt = examplePrompt


short = QuestionType(
    "short", "You must generate short questions that are clear to understand.")
long = QuestionType(
    "long", "You must generate long answer questions. Examples include essay questions.")
mcq = QuestionType(
    "mcq", "You must generate short multiple choice questions that can be answered using one sentence. Do not return the answers or options.", )
ALLQUESTIONTYPES: dict[str, QuestionType] = {}

ALLQUESTIONTYPES[short.name] = short
ALLQUESTIONTYPES[long.name] = long
ALLQUESTIONTYPES[mcq.name] = mcq


if QUESTIONTYPE not in ALLQUESTIONTYPES.keys():
    raise Exception("Invalid question type.")

In [7]:
"""Modify the notes such that they are in a structure that is best for the question type."""
NOTESREWRITE = f"Instruct: Rewrite, reorganise and summmarise these notes below into one simple clear paragraph. Make sentences simple. Do not add any other information. Do not write the questions. Write each note in a definition-example-explanation structure. Make sure that the data is all in plain-text. \n \n {NOTES} \n \n \nOutput: " 

print(NOTESREWRITE)

rewriteNotes = ollama.generate(model=MODELNAME, prompt=NOTESREWRITE)

print(rewriteNotes["response"])

NOTES = rewriteNotes["response"]

Instruct: Rewrite, reorganise and summmarise these notes below into one simple clear paragraph. Make sentences simple. Do not add any other information. Do not write the questions. Write each note in a definition-example-explanation structure. Make sure that the data is all in plain-text. 
 
 A distributed system is a collection of autonomous computers linked by a network, with software to produce an integrated computing infrastructure. 
It is a set of discrete computers that perform a computation together, as if they were a single computing system, as a network of processes, which interact with one another to achieve a goal. These systems are required for processes like resource sharing, computation speed up via parallelism, and fault tolerance and uncertainty management. 
Information is processed data; data that is organised, meaningful + useful. Security is when you prevent unauthorised access to information + services, and how to maintain availability of information + services to a

In [8]:
""" The system prompt should be designed to use few-shot learning in order to generate the most accurate questions for the source material. """


SYSTEMPROMPT = f"Instruct: Generate {QUESTIONNUMBER} questions, in a list. Each question should be written as a string, and should be separated by one new line character \\n each. Questions should not be numbered. \nFor example, for 4 questions about famous authors would be generated as\n \"Which author beginning with the letter M contributed the most to American literary fiction?\"\\n\"When was this author born?\"\\n\"Which famous comet was associated with this author?\"\\n\"Did this author have a pen-name?\"\n "

if ALLQUESTIONTYPES[QUESTIONTYPE].examplePrompt != None:
  SYSTEMPROMPT += ALLQUESTIONTYPES[QUESTIONTYPE].systemPrompt + ALLQUESTIONTYPES[QUESTIONTYPE].examplePrompt
else:
  SYSTEMPROMPT += ALLQUESTIONTYPES[QUESTIONTYPE].systemPrompt

"""Adding notes specified by user."""
# if EXAMPLEQUESTIONS != "":
#     SYSTEMPROMPT += f"You must generate questions with a similar structure and form to the questions below: \n {EXAMPLEQUESTIONS}"
SYSTEMPROMPT += f"\nPlease generate the questions from these notes: \n {NOTES} \n Output: "
print(SYSTEMPROMPT)

Instruct: Generate 5 questions, in a list. Each question should be written as a string, and should be separated by one new line character \n each. Questions should not be numbered. 
For example, for 4 questions about famous authors would be generated as
 "Which author beginning with the letter M contributed the most to American literary fiction?"\n"When was this author born?"\n"Which famous comet was associated with this author?"\n"Did this author have a pen-name?"
 You must generate long answer questions. Examples include essay questions.
Please generate the questions from these notes: 
  A distributed system is an assembly of autonomous computers connected through a network, operating as one cohesive unit to accomplish tasks like resource sharing and increasing computation speed through parallelism, while managing fault tolerance and uncertainty. These systems utilize interprocess communications, involving networking layers such as the TCP protocol for reliable communication. Middlew

In [9]:
generation = ollama.generate(model=MODELNAME, prompt=SYSTEMPROMPT)
# print(generation["response"])
generated_questions = [question.replace(
    "\"", "") for question in generation["response"].split("\\n")]
# Filter out the blank spaces.
generated_questions = [question.strip() for question in generated_questions if question != ""]

# print(generated_questions)
for q in range(len(generated_questions)):
  print(f"Question [{q+1}] {generated_questions[q]}")


if len(generated_questions) != QUESTIONNUMBER:
  raise Exception(f"Problem with generated question amount: expected {QUESTIONNUMBER}, got {len(generated_questions)}")
 

Question [1] How does the TCP protocol contribute to reliable communication within distributed systems, particularly concerning interprocess communications?
Question [2] In what ways do middleware tools like Java RMI enhance remote procedure calls in distributed computing environments, and how do they abstract away from implementation details for developers?
Question [3] Can you elaborate on the role of data marshalling in facilitating cross-platform method invocations within a Remote Method Invocation (RMI) framework, including its impact on system performance?
Question [4] How does Java's `java.security.policy` mechanism ensure security and manage permissions across distributed systems with diverse networked resources?
Question [5] What are the advantages of accessing object specifications through both local file systems and HTTP web servers in a distributed computing environment, and how do these methods differ in their approach to resource sharing and accessibility?


In [10]:
if QUESTIONTYPE == 'mcq':
  """ Generate choices for each question."""
  MCQOPTIONS = 4
  QUESTIONDICT = {}
  for question in generated_questions:
    CHOICEPROMPT = f"Generate a short correct answer, that is less than 10 words, to the question {question}. This answer should be summarised and should not include any notes or formatting. \nFor example: for the question \"What is the capital of England?\" the output would be \"London\". \nFor example: for the question \"How many seas are there in the world\" the output would be \"7\". \nFor example: for the question \"What is an example of a JavaScript framework?\" the output would be \"React.js\""
    # CHOICEPROMPT = f"Please generate {MCQOPTIONS} different multiple choice answers for the question {question}. Each choice must be separated by a new line character \\n. \nFor example, for the question: What is the capital of Germany? \n The output should be: Cologne\\nFrankfurt\\nBerlin\\nHamburg. \nFor the question: How many moons does Earth have? The output could be: 1\\n2\\n4\\n3. Each choice should not be in a numbered list."
    # CHOICEPROMPT += f"Whenever possible, take the answer from the notes \n{NOTES}\n, but the answer should not be vague. "
    correctChoiceGeneration = ollama.generate(
        model=MODELNAME, prompt=CHOICEPROMPT)
    """Output Parser"""
    choiceList = [choice.replace(
        "\"", "") for choice in correctChoiceGeneration["response"].split("\\n") if choice != ""]
    QUESTIONDICT[str(question)] = choiceList
    print(question)
    print(choiceList)
    incorrectPrompt = f"Generate {MCQOPTIONS - 1} options, that are similar but not the same as {choiceList[0]}. \nFor example: for an option London, the output would be \"Edinburgh\\nGlasgow\\nAberdeen\\Leeds\". \nFor example: for an option \"7\" the output would be \"4\\n6\\n2\\n9\". \n"
    incorrectChoiceGeneration = ollama.generate(
        model=MODELNAME, prompt=incorrectPrompt)
    print(incorrectChoiceGeneration["response"].split("\\n"))
    
  
  

In [11]:
""" Add generated questions to a tex file."""
try:
  with open("questions.tex", "w") as questionTemplate:
      if QUESTIONTYPE == "short":
        # my_file.write("\\begin{parts}\n")
        for question in generated_questions:
          questionTemplate.write(
              f"\\question[{TOTALMARKS // QUESTIONNUMBER}] {question} \n")
          questionTemplate.write("\\fillwithlines{0.75in}")
          # for i in range(5):
          #   my_file.write(f"\\newline")
          #   my_file.write(f"{{\\rule{{\\linewidth}}{{0.5pt}}}} \n")
          #   my_file.write(f"\\newline")
          questionTemplate.write(f"\\vspace{{0.5in}}")
          # [my_file.write("\\choice {choices}") for choice in choices]
          # my_file.write(f"\n")
        # my_file.write("\\end{parts}")
      elif QUESTIONTYPE == "mcq":   
        # my_file.write("\\begin{parts}\n")
        for question in generated_questions:  
          questionTemplate.write(
              f"\\question[{TOTALMARKS // QUESTIONNUMBER}] {question} \n")
          questionTemplate.write("\\begin{checkboxes} \n")
          # [my_file.write("\\choice {choices}") for choice in choices]
          questionTemplate.write("\\end{checkboxes} \n")
          # my_file.write(f"\n")
        # my_file.write("\\end{parts}")
      elif QUESTIONTYPE == "long":
        # my_file.write("\\begin{parts}\n")
        for question in generated_questions:
          questionTemplate.write(
              f"\\question[{TOTALMARKS // QUESTIONNUMBER}] {question} \n")
          # for i in range(10):
          #   my_file.write(f"\\newline")
          #   my_file.write(f"{{\\rule{{\\linewidth}}{{0.5pt}}}} \n")
          #   my_file.write(f"\\newline")
          questionTemplate.write("\\fillwithlines{2in}")
          questionTemplate.write(f"\\vspace{{0.5in}}")
          
      else:
        raise Exception("Behavior for questions not implemented.")
except OSError:
   raise Exception("questions.tex does not exist and cannot be written to.")

In [12]:
def compileTeX(texFile:str):
  """Uses pdflatex to compile the file."""
  if texFile[-3:] == "tex":
    subprocess.run(["pdflatex", "-interaction", "nonstopmode", texFile])
  else:
    raise Exception("This is not a tex file and cannot be compiled.")

compileTeX("exam_1.tex")

This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./exam_1.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-02-20>
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/exam/exam.cls
Document Class: exam 2023/07/09 Version 2.704 by Philip Hirschhorn
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/base/ifthen.sty)
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/base/size10.clo)))
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/l3backend/l3backend-pdftex.d
ef) (./exam_1.aux) (./questions.tex [1{/usr/local/texlive/2024basic/texmf-var/f
onts/map/pdftex/updmap/pdftex.map}] [2]) [3]
This exam contains 5 questions with 0 parts, 0 subparts, and 0 subsubparts.
This exam has a total of 10 points.
This exam has a total of 0 bonus points.
(./ex

In [17]:
"""Create Streamlit app."""

st.title("Magister")

2024-06-13 14:19:22.112 
  command:

    streamlit run /opt/homebrew/Caskroom/miniconda/base/envs/lmpc/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()